In [28]:
%load_ext autotime
import geopandas as gpd
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import statsmodels.api as sm
import shapely
from tqdm.contrib.concurrent import process_map
from glob import glob
pd.set_option('display.max_columns', None)

In [29]:
shorelines = gpd.read_file("Data for testing/NZCCDv1.shp")
shorelines

,Region,Site,Digitiser,Scale,Notes,Source,CPS,Proxy,Photoscale,Georef_ER,Pixel_Er,Total_UNCY,USDate,SHLength,Date,ID,geometry
0,Auckland,KarekareBethells,MW,2000,tod,RL,4,1,40000,5.03,1.416425,5.620681,01/02/2004,1.265403,2004-01-02,0.0,"LINESTRING Z (1728907.5 5916213.248 0, 1728868..."
1,Auckland,KarekareBethells,MW,2000,tod,RL,4,1,40000,5.03,1.416425,5.620681,01/02/2004,0.307010,2004-01-02,1.0,"LINESTRING Z (1729067.838 5914779.733 0, 17290..."
2,Auckland,KarekareBethells,MW,2000,tod (landward),RL,4,1,40000,5.03,1.416425,5.620681,01/02/2004,0.255243,2004-01-02,2.0,"LINESTRING Z (1729647.881 5911983.994 0, 17296..."
3,Auckland,KarekareBethells,MW,2000,tod,RL,3,1,40000,5.03,1.416425,5.314890,01/02/2004,1.107251,2004-01-02,3.0,"MULTILINESTRING Z ((1728131.079 5917218.888 0,..."
4,Auckland,KarekareBethells,MW,2000,tod,RL,4,1,40000,5.03,1.416425,5.620681,01/02/2004,0.770375,2004-01-02,4.0,"MULTILINESTRING Z ((1729643.228 5912906.682 0,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19661,West Coast,WoodpeckerBay,MD,1000,EdgeVeg,RL,2,1,18800,4.00,0.630693,3.736680,11/18/1948,1.191368,1948-11-18,19661.0,"LINESTRING Z (1467237.671 5348414.193 0, 14672..."
19662,Otago,Oamaru,TK,1500,Top of cliff,Max,3,2,0,2.02,0.412236,2.278429,04/04/2018,0.650952,2018-04-04,19662.0,"LINESTRING Z (1444386.944 5008247.367 0, 14443..."
19663,Otago,Oamaru,TK,1500,Top of cliff,Max,3,2,0,2.02,0.412236,2.278429,04/04/2018,0.179434,2018-04-04,19663.0,"LINESTRING Z (1444860.629 5008686.925 0, 14448..."
19664,Otago,Oamaru,TK,1500,Top of cliff,Max,3,2,0,2.02,0.412236,2.278429,04/04/2018,0.174825,2018-04-04,19664.0,"LINESTRING Z (1444996.286 5008822.252 0, 14450..."


In [30]:
transects = pd.concat(gpd.read_file(f).set_index("Unique_ID") for f in glob("Data for testing/Unique_ID_transects/*.shp"))
transects

/home/nyou045/retrolens/venv/lib/python3.10/site-packages/pyogrio/raw.py:196: UserWarning: Measured (M) geometry types are not supported. Original type 'Measured 3D LineString' is converted to 'LineString Z'
  return ogr_read(
/home/nyou045/retrolens/venv/lib/python3.10/site-packages/pyogrio/raw.py:196: UserWarning: Measured (M) geometry types are not supported. Original type 'Measured 3D LineString' is converted to 'LineString Z'
  return ogr_read(
/home/nyou045/retrolens/venv/lib/python3.10/site-packages/pyogrio/raw.py:196: UserWarning: Measured (M) geometry types are not supported. Original type 'Measured 3D LineString' is converted to 'LineString Z'
  return ogr_read(
/home/nyou045/retrolens/venv/lib/python3.10/site-packages/pyogrio/raw.py:196: UserWarning: Measured (M) geometry types are not supported. Original type 'Measured 3D LineString' is converted to 'LineString Z'
  return ogr_read(
/home/nyou045/retrolens/venv/lib/python3.10/site-packages/pyogrio/raw.py:196: UserWarning: M

,geometry
Unique_ID,
1.005226e+11,"LINESTRING Z (1680443.328 5669799.874 0, 16804..."
1.005226e+11,"LINESTRING Z (1680451.296 5669793.831 0, 16804..."
1.005226e+11,"LINESTRING Z (1680459.263 5669787.788 0, 16804..."
1.005227e+11,"LINESTRING Z (1680467.271 5669781.803 0, 16804..."
1.005227e+11,"LINESTRING Z (1680475.694 5669776.413 0, 16804..."
...,...
2.020008e+11,"LINESTRING Z (1616477.032 5429332.43 0, 161634..."
2.020008e+11,"LINESTRING Z (1616467.288 5429330.182 0, 16163..."
2.020008e+11,"LINESTRING Z (1616457.544 5429327.935 0, 16163..."


In [31]:
transects.geometry.apply(lambda g: len(g.coords)).value_counts()

geometry
2    228070
3       543
Name: count, dtype: int64

In [32]:
rates = gpd.read_file("kx-nzccd-coastal-change-rates-SHP.zip")
rates

,ShrCount,LRR,LR2,LCI,LSE,EPR,EPRunc,NSM,SCE,WLR,WR2,WCI,WSE,TCD,Region,UniqueID,Start_date,End_date,Duration,geometry
0,13.0,-0.27,0.80,0.07,3.20,-0.27,0.05,-19.30,24.84,-0.23,0.65,0.09,1.37,4015.900000,Waikato,1.005336e+11,1950-04-01,2022-02-15,71,POINT (1740823.984 5718945.283)
1,9.0,0.58,0.91,0.13,5.14,0.56,0.07,39.42,39.42,0.55,0.93,0.10,1.14,44129.150000,Tasman,2.000050e+11,1950-10-11,2021-03-13,70,POINT (1573804.234 5506243.949)
2,13.0,-0.16,0.07,0.32,16.77,-0.07,0.06,-5.38,44.09,-0.19,0.08,0.34,4.94,7410.848139,Tasman,2.010001e+11,1940-02-22,2020-07-01,80,POINT (1602827.257 5444493.261)
3,13.0,-0.22,0.13,0.30,15.96,-0.12,0.06,-9.85,44.06,-0.25,0.15,0.32,4.70,7420.848139,Tasman,2.010001e+11,1940-02-22,2020-07-01,80,POINT (1602821.392 5444501.389)
4,13.0,-0.26,0.19,0.28,15.15,-0.18,0.06,-14.58,44.19,-0.29,0.21,0.30,4.41,7430.848139,Tasman,2.010001e+11,1940-02-22,2020-07-01,80,POINT (1602815.442 5444509.464)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228608,5.0,0.01,0.05,0.08,1.81,-0.02,0.07,-1.50,3.73,0.00,0.00,0.13,0.97,3223.050000,Auckland,1.010019e+11,1961-11-02,2022-12-21,61,POINT (1784449.725 5926935.288)
228609,5.0,0.06,0.63,0.07,1.45,0.03,0.07,1.91,4.73,0.05,0.36,0.09,0.71,3233.050000,Auckland,1.010019e+11,1961-11-02,2022-12-21,61,POINT (1784439.931 5926939.809)
228610,5.0,0.06,0.64,0.06,1.27,0.03,0.07,1.74,4.23,0.07,0.46,0.10,0.74,3243.050000,Auckland,1.010019e+11,1961-11-02,2022-12-21,61,POINT (1784431.172 5926944.695)
228611,5.0,0.03,0.34,0.06,1.26,0.00,0.07,0.18,2.73,0.04,0.22,0.09,0.70,3253.050000,Auckland,1.010019e+11,1961-11-02,2022-12-21,61,POINT (1784423.016 5926950.491)


In [33]:
rates.Region.value_counts()

Region
Northland      37274
Canterbury     21881
WestCoast      20990
Taranaki       17050
Waikato        16589
Otago          15425
BayOfPlenty    13902
HawkesBay      13494
Southland      12382
Auckland       11494
Wellington     11154
Manawatu       10443
Tasman         10102
Gisborne        8314
Marlborough     5083
Nelson          3036
Name: count, dtype: int64

In [34]:
def get_timeseries(Unique_ID):
    transect = transects.geometry[Unique_ID]
    shorelines["intersect_point"] = shorelines.intersection(transect)
    df = shorelines[~shorelines.intersect_point.is_empty].sort_values("Date")
    df.Date = pd.to_datetime(df.Date)
    df["YearsSinceBase"] = (df.Date - pd.Timestamp(1800, 1, 1)).dt.days / 365.25
    transect_origin = shapely.get_point(transect, -1)
    df["Distance"] = df.intersect_point.distance(transect_origin)
    return {
        "UniqueID": Unique_ID,
        "Dates": df.Date.dt.strftime("%Y-%m-%d").tolist(),
        "Distances": df.Distance.round(2).tolist(),
    }
timeseries = process_map(get_timeseries, transects.index, chunksize=100, max_workers=64)

100%|██████████| 228613/228613 [03:45<00:00, 1015.67it/s]


In [35]:
rates_with_timeseries = rates.merge(pd.DataFrame(timeseries), on="UniqueID")
rates_with_timeseries["lat"] = rates_with_timeseries.geometry.to_crs(4326).y
rates_with_timeseries["lon"] = rates_with_timeseries.geometry.to_crs(4326).x
rates_with_timeseries.drop(columns="geometry").to_parquet("rates_with_timeseries.parquet")

In [36]:
rates_with_timeseries

,ShrCount,LRR,LR2,LCI,LSE,EPR,EPRunc,NSM,SCE,WLR,WR2,WCI,WSE,TCD,Region,UniqueID,Start_date,End_date,Duration,geometry,Dates,Distances,lat,lon
0,13.0,-0.27,0.80,0.07,3.20,-0.27,0.05,-19.30,24.84,-0.23,0.65,0.09,1.37,4015.900000,Waikato,1.005336e+11,1950-04-01,2022-02-15,71,POINT (1740823.984 5718945.283),"[1950-04-01, 1974-09-12, 1979-04-01, 1986-11-2...","[157.05, 151.78, 144.95, 145.67, 138.57, 135.4...",-38.666886,174.618687
1,9.0,0.58,0.91,0.13,5.14,0.56,0.07,39.42,39.42,0.55,0.93,0.10,1.14,44129.150000,Tasman,2.000050e+11,1950-10-11,2021-03-13,70,POINT (1573804.234 5506243.949),"[1950-10-11, 1967-10-10, 1972-10-29, 1980-01-3...","[381.7, 382.28, 396.97, 406.57, 413.32, 421.12...",-40.594191,172.690413
2,13.0,-0.16,0.07,0.32,16.77,-0.07,0.06,-5.38,44.09,-0.19,0.08,0.34,4.94,7410.848139,Tasman,2.010001e+11,1940-02-22,2020-07-01,80,POINT (1602827.257 5444493.261),"[1940-02-22, 1947-03-27, 1958-05-19, 1967-02-0...","[299.72, 327.27, 330.16, 324.96, 329.02, 335.3...",-41.150878,173.033694
3,13.0,-0.22,0.13,0.30,15.96,-0.12,0.06,-9.85,44.06,-0.25,0.15,0.32,4.70,7420.848139,Tasman,2.010001e+11,1940-02-22,2020-07-01,80,POINT (1602821.392 5444501.389),"[1940-02-22, 1947-03-27, 1958-05-19, 1967-02-0...","[305.24, 330.16, 329.59, 324.55, 329.52, 335.5...",-41.150805,173.033624
4,13.0,-0.26,0.19,0.28,15.15,-0.18,0.06,-14.58,44.19,-0.29,0.21,0.30,4.41,7430.848139,Tasman,2.010001e+11,1940-02-22,2020-07-01,80,POINT (1602815.442 5444509.464),"[1940-02-22, 1947-03-27, 1958-05-19, 1967-02-0...","[309.71, 331.19, 329.28, 324.13, 329.9, 335.78...",-41.150732,173.033553
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228608,5.0,0.01,0.05,0.08,1.81,-0.02,0.07,-1.50,3.73,0.00,0.00,0.13,0.97,3223.050000,Auckland,1.010019e+11,1961-11-02,2022-12-21,61,POINT (1784449.725 5926935.288),"[1961-11-02, 1980-10-24, 2010-11-01, 2017-04-2...","[168.17, 166.35, 170.08, 169.24, 166.66]",-36.785447,175.067104
228609,5.0,0.06,0.63,0.07,1.45,0.03,0.07,1.91,4.73,0.05,0.36,0.09,0.71,3233.050000,Auckland,1.010019e+11,1961-11-02,2022-12-21,61,POINT (1784439.931 5926939.809),"[1961-11-02, 1980-10-24, 2010-11-01, 2017-04-2...","[163.56, 162.46, 167.19, 166.85, 165.47]",-36.785409,175.066994
228610,5.0,0.06,0.64,0.06,1.27,0.03,0.07,1.74,4.23,0.07,0.46,0.10,0.74,3243.050000,Auckland,1.010019e+11,1961-11-02,2022-12-21,61,POINT (1784431.172 5926944.695),"[1961-11-02, 1980-10-24, 2010-11-01, 2017-04-2...","[161.15, 161.6, 164.56, 165.38, 162.89]",-36.785366,175.066894
228611,5.0,0.03,0.34,0.06,1.26,0.00,0.07,0.18,2.73,0.04,0.22,0.09,0.70,3253.050000,Auckland,1.010019e+11,1961-11-02,2022-12-21,61,POINT (1784423.016 5926950.491),"[1961-11-02, 1980-10-24, 2010-11-01, 2017-04-2...","[161.38, 161.33, 163.65, 164.06, 161.56]",-36.785316,175.066802


In [41]:
rates[rates.UniqueID == 100495183848.0]

,ShrCount,LRR,LR2,LCI,LSE,EPR,EPRunc,NSM,SCE,WLR,WR2,WCI,WSE,TCD,Region,UniqueID,Start_date,End_date,Duration,geometry
135428,5.0,-1.2,0.46,1.77,11.24,-1.05,0.65,-19.84,36.49,-1.18,0.44,1.82,1.33,3916.265678,Wellington,1.004952e+11,2002-04-11,2021-02-13,18,POINT (1777488.925 5486087.044)


In [43]:
transect = transects.geometry[100495183848.0]
shorelines["intersect_point"] = shorelines.intersection(transect)
shorelines[~shorelines.intersect_point.is_empty].sort_values("Date")

,Region,Site,Digitiser,Scale,Notes,Source,CPS,Proxy,Photoscale,Georef_ER,Pixel_Er,Total_UNCY,USDate,SHLength,Date,ID,geometry,intersect_point
18386,Wellington,Otaki,ER,1000,None,LDS,5,1,0,0.0,0.3,8.595237,02/13/2021,0.339523,2021-02-13,18386.0,"LINESTRING Z (1777526.996 5486043.881 0, 17775...",POINT Z (1777532.001 5486065.092 0)


In [50]:
m = shorelines[shorelines.Site == "Otaki"].explore()
transects[100495183848.0:100495183848.0].explore(m=m, color="red")

In [51]:
def process_row(Unique_ID):
    transect = transects.geometry[Unique_ID]
    # This function reproduces the statistics described in https://pubs.usgs.gov/of/2021/1091/ofr20211091.pdf
    shorelines["intersect_point"] = shorelines.intersection(transect)
    df = shorelines[~shorelines.intersect_point.is_empty].sort_values("Date")
    if len(df) < 3:
        print(f"{Unique_ID} has only {len(df)} shorelines")
        return {}
    df.Date = pd.to_datetime(df.Date)
    df["YearsSinceBase"] = (df.Date - pd.Timestamp(1800, 1, 1)).dt.days / 365.25
    transect_origin = shapely.get_point(transect, -1)
    df["Distance"] = df.intersect_point.distance(transect_origin)
    LR = sm.OLS(df.Distance, sm.add_constant(df.YearsSinceBase)).fit()
    low, high = LR.conf_int(alpha=.1).loc["YearsSinceBase"]
    LCI = (high - low) / 2
    WLR = sm.WLS(df.Distance, sm.add_constant(df.YearsSinceBase), weights=1 / df.Total_UNCY ** 2).fit()
    low, high = WLR.conf_int(alpha=.1).loc["YearsSinceBase"]
    WCI = (high - low) / 2
    duration = (df.Date.max() - df.Date.min()).days / 365.25
    assert duration > 0, f"{Unique_ID} has a duration of {duration}"
    NSM = df.intersect_point.iloc[0].distance(df.intersect_point.iloc[-1])
    return {
        "UniqueID": Unique_ID,
        "Region": df.Region.value_counts().idxmax(),
        "Start_date": str(df.Date.min().date()),
        "End_date": str(df.Date.max().date()),
        "Duration": round(duration),
        "ShrCount": len(df),
        "NSM": round(NSM, 2),
        "SCE": df.intersect_point.apply(lambda p: df.intersect_point.distance(p).max()).max().round(2),
        "EPR": round(NSM / duration, 2),
        "EPRunc": round(np.sqrt(df.Total_UNCY.iloc[0] ** 2 + df.Total_UNCY.iloc[-1] ** 2) / duration, 2),
        "LRR": LR.params.YearsSinceBase.round(2),
        "LCI": round(LCI, 2),
        "LSE": np.sqrt(LR.mse_resid).round(2),
        "LR2": LR.rsquared.round(2),
        "WLR": WLR.params.YearsSinceBase.round(2),
        "WCI": round(WCI, 2),
        "WSE": np.sqrt(WLR.mse_resid).round(2),
        "WR2": WLR.rsquared.round(2),
        "geometry": transect,
        "Dates": df.Date.dt.strftime("%Y-%m-%d").tolist(),
        "Distances": df.Distance.round(2).tolist(),
    }

transects = process_map(process_row, transects.index, chunksize=100, max_workers=64)
transects

  0%|          | 0/228613 [00:00<?, ?it/s]

100260096672.0 has only 2 shorelines


  0%|          | 901/228613 [00:10<23:24, 162.16it/s] 

100191251944.0 has only 2 shorelines
100586128441.0 has only 2 shorelines
100586130639.0 has only 2 shorelines


 11%|█         | 24601/228613 [00:33<01:41, 2002.69it/s]

...

101011595606.0 has only 2 shorelines


 98%|█████████▊| 223688/228613 [04:17<00:04, 1010.49it/s]

200247583487.0 has only 2 shorelines


100%|██████████| 228613/228613 [04:20<00:00, 877.81it/s] 


[{'UniqueID': 100522647132.0,
  'Region': 'Taranaki',
  'Start_date': '1970-02-10',
  'End_date': '2022-01-14',
  'Duration': 52,
  'ShrCount': 5,
  'NSM': 15.55,
  'SCE': np.float64(16.84),
  'EPR': 0.3,
  'EPRunc': np.float64(0.2),
  'LRR': np.float64(0.24),
  'LCI': 0.37,
  'LSE': np.float64(6.55),
  'LR2': np.float64(0.43),
  'WLR': np.float64(0.28),
  'WCI': 0.64,
  'WSE': np.float64(2.1),
  'WR2': np.float64(0.26),
  'geometry': <LINESTRING Z (1680443.328 5669799.874 0, 1680443.328 5669799.874 0, 1680262...>,
  'Dates': ['1970-02-10',
   '2000-01-14',
   '2011-03-11',
   '2016-10-31',
   '2022-01-14'],
  'Distances': [255.9, 263.3, 254.61, 268.75, 271.45]},
 {'UniqueID': 100522648133.0,
  'Region': 'Taranaki',
  'Start_date': '1970-02-10',
  'End_date': '2022-01-14',
  'Duration': 52,
  'ShrCount': 5,
  'NSM': 12.63,
  'SCE': np.float64(18.04),
  'EPR': 0.24,
  'EPRunc': np.float64(0.2),
  'LRR': np.float64(0.13),
  'LCI': 0.41,
  'LSE': np.float64(7.16),
  'LR2': np.float64(0.16

In [ ]:
points = gpd.read_file("Data for testing/Unique_ID_points/NorthIsland_UniqueID.shp")
points

,Unique_ID,geometry
0,1.003479e+11,POINT Z (1880322.836 5830555.229 0)
1,1.003479e+11,POINT Z (1880327.712 5830546.382 0)
2,1.003479e+11,POINT Z (1880333.202 5830537.997 0)
3,1.003479e+11,POINT Z (1880339.081 5830529.906 0)
4,1.003479e+11,POINT Z (1880345.252 5830522.035 0)
...,...,...
139133,1.004750e+11,POINT Z (1758086.752 5426055.275 0)
139134,1.004750e+11,POINT Z (1758091.623 5426064.015 0)
139135,1.004750e+11,POINT Z (1758096.813 5426072.563 0)
139136,1.004750e+11,POINT Z (1758102.158 5426081.017 0)
